# Implement of SVM, naive bayes and random forest

In [17]:
import os
import re
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import train_test_split, GridSearchCV

%matplotlib inline

In [4]:
df = pd.read_csv('cleaned_data.csv', index_col=0)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
df.tail()

,content,label,content_cut
86858,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,1,新华社 照片 多伦多 2017年 6月 7日 体育 2 冰球 国家 女子 冰球队 海外 选秀...
86859,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,1,新华社 兰州 6月 3日 电 王衡 徐丹 记者 从 甘肃省 交通 运输厅 获悉 甘肃 近日 ...
86860,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,1,n2017 年 5月 29日 在 法国 巴黎 郊外 的 凡尔赛宫 法国 总统 马克龙 出席 ...
86861,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,1,n2017 年 5月 25日 在 美国 马萨诸塞州 剑桥市 哈佛 大学 毕业生 在 毕业 典...
86862,新华社德国杜塞尔多夫６月６日电题：乒乓女球迷 \n 新华社记者王子江、张寒\n 熊老...,1,新华社 德国 杜塞尔多夫 ６月 ６日 电 题 乒乓 女 球迷 新华社 记者 王子江 张寒 熊...


In [6]:
df.label.value_counts()

1    78472
0     8391
Name: label, dtype: int64

# Handle imbalanced datasets

正负类数据比大概在10:1，因此需要考虑数据偏斜带来的影响，常见处理方法：

1. 数据角度
    1. 搜集更多偏少类数据使数据集平衡；
    2. 过采样和欠采样；
    3. SMOTE等合成新样本；
    4. 增加样本较少类的权重
2. 评价角度
    1. 采用F1衡量对正负类的分类结果；
    2. 谨慎选择AUC作为评价指标，观察更多的PR曲线
3. 算法角度
    1. 数据比较极端的情况，可转换成异常检测问题
    2. 集成学习 -- 加法模型，adaboost，随机森林
    3. 选择对数据偏斜不敏感的模型，如树模型



## Vectorized

In [7]:
vectorized = TfidfVectorizer(max_features=800)

In [8]:
X = vectorized.fit_transform(df.content_cut)

In [9]:
X = X.toarray()
y = df.label.to_numpy()

In [10]:
X.shape, y.shape

((86863, 800), (86863,))

* scipy库中consine的实现为：$$cosine(u, v) = 1 - \frac{u \cdot v}{||u||_2 ||v||_2}$$  取值范围[0, 2];因此这个函数的结果越小说明越相似。

In [11]:
def corpus_split(X, y):
    Xtrain,Xtest, ytrain, ytest = train_test_split(X, y, test_size=.2)
    Xtrain, Xcv, ytrain, ycv = train_test_split(Xtrain, ytrain, test_size=.25)
    
    return Xtrain,ytrain, Xcv, ycv, Xtest , ytest

In [12]:
Xtrain, ytrain, Xcv, ycv, Xtest, ytest = corpus_split(X, y)

In [13]:
Xtrain.shape,ytrain.shape, Xcv.shape, ycv.shape, Xtest.shape , ytest.shape

((52117, 800), (52117,), (17373, 800), (17373,), (17373, 800), (17373,))

## SVM

In [66]:
svc = SVC(gamma='auto')
svc.fit(Xtrain, ytrain)
svc.score(Xcv, ycv)

0.9009382374949635

In [67]:
ypred = svc.predict(Xtest)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1586
           1       0.91      1.00      0.95     15787

    accuracy                           0.91     17373
   macro avg       0.45      0.50      0.48     17373
weighted avg       0.83      0.91      0.87     17373



C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


slow and unacceptable.

### SVM without kernel

In [81]:
svc = SVC(kernel='linear', gamma='auto')
svc.fit(Xtrain, ytrain)
svc.score(Xcv, ycv)

0.9812928106832441

In [67]:
ypred = svc.predict(Xtest)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1586
           1       0.91      1.00      0.95     15787

    accuracy                           0.91     17373
   macro avg       0.45      0.50      0.48     17373
weighted avg       0.83      0.91      0.87     17373



C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


didn't change much.

## Naive Bayes

In [14]:
gnb = GNB()
gnb.fit(Xtrain, ytrain)
gnb.score(Xcv, ycv)

0.8712945375007195

In [15]:
ypred = gnb.predict(Xtest)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.44      0.94      0.60      1684
           1       0.99      0.87      0.93     15689

    accuracy                           0.88     17373
   macro avg       0.71      0.91      0.76     17373
weighted avg       0.94      0.88      0.89     17373



poor precision when true positive is 0 and not good enough recall when true positive is 1.

### some other metrics
Default positive label is 1, so the following score only show you model's performance on 1.

In [18]:
print(f1_score(ytest, ypred))
print(precision_score(ytest, ypred))
print(recall_score(ytest, ypred))
print(roc_auc_score(ytest, ypred))

0.9269254299503773
0.9929367217650914
0.8691439862323921
0.9057715180568136


while positive label is 0.

In [21]:
print(f1_score(ytest, ypred, pos_label=0))
print(precision_score(ytest, ypred, pos_label=0))
print(recall_score(ytest, ypred, pos_label=0))

0.5961682945154019
0.435989010989011
0.9423990498812351


## Decision Tree

In [70]:
dtc = DTC()
dtc.fit(Xtrain, ytrain)

In [72]:
dtc.score(Xcv, ycv)

0.9898117768951822

In [71]:
ypred = dtc.predict(Xtest)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1586
           1       0.99      0.99      0.99     15787

    accuracy                           0.99     17373
   macro avg       0.97      0.97      0.97     17373
weighted avg       0.99      0.99      0.99     17373



not bad.

## Random Forest

In [79]:
rfc = RFC(n_estimators=100)
rfc.fit(Xtrain, ytrain)
rfc.score(Xcv, ycv)

0.9924595636907846

In [80]:
ypred = rfc.predict(Xtest)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      1586
           1       1.00      1.00      1.00     15787

    accuracy                           0.99     17373
   macro avg       0.98      0.98      0.98     17373
weighted avg       0.99      0.99      0.99     17373



We have a winner!

## Advantages and disadvantages of classic machine learning methods
- Linear Regression
    - 优点
        - 简单，直观
    - 缺点
        - 无法拟合非线性数据以及稍复杂模式
- Logistic Regression
    - 优点
        - 直观，结果有很好的概率解释，可通过正则化避免过拟合
    - 缺点
        - 多分类以及非线性决策边界的表现一般
- KNN
    - 优点
        - 训练集，距离度量方法和K值以及决策规则确定后，结果唯一
    - 缺点
        - 需要考虑如何搜索最近K个近邻点，一般的范式距离过于消耗储存空间和计算时间
- Naive Bayes
    - 优点
        - 简单
    - 缺点
        - 核心假设：各特征值之间条件独立在现实中难以成立
- SVM
    - 应用场景
        - 特征量大而训练集中等时，采用Logistics或者线性SVM
        - 特征量小而训练集大时，采用GaussianKernel SVM
    - 优点
        - 根据数据情况可选择是否采用内核函数进行模型训练，对于非线性决策边界的数据表现较好
        - 可选内核函数，可有效避免过拟合
        - 高维空间的模型训练表现比其他模型更好
    - 缺点
        - 可选内核函数较多，导致参数调整过于复杂
        - 计算消耗大

- Decision Tree
    - 优点
        - 计算量简单，可解释性强，适合处理缺失属性值的样本，可拟合非线性训练集，对奇异数据不敏感
    - 缺点
        - 易过拟合

- Random Forest
    - 优点
        - 在当前的很多数据集上，相对其他算法有着很大的优势，表现良好
        - 它能够处理很高维度的数据，并且不用做特征选择，因为特征子集是随机选择的
        - 在训练完后，它能够得出特征重要性
        - 在创建随机森林的时候，对generlization error使用的是无偏估计，模型泛化能力强
        - 随机森林有oob，不需要单独换分交叉验证集
        - 训练时树与树之间是相互独立的，训练速度快，容易做成并行化方法
        - 对缺失值不敏感，如果有很大一部分的特征遗失，仍可以维持准确度。
    - 缺点
        - 随机森林在某些噪音较大的分类或回归问题上会过拟合
        - 对于有不同取值的属性的数据，取值划分较多的属性会对随机森林产生更大的影响
- K-Means
    - 优点
        - 计算时间复杂度为O(NKt)接近于线性，相对高效，其中N为训练集数量，K为聚类中心数，t为迭代次数，
    - 缺点
        - 易受噪点影响， 通常需要预处理排除奇异值导致簇中心偏移的干扰
        - 需要合理选择k值
        - 受初始化的K个簇中心位置的影响较大，结果可能因每次初始化的中心点不同而不同，难以得到最优解
        - 对数据偏斜较大（如不同类数量差在百倍以上）的数据支持不是很好
